In [27]:
# imports
import numpy as np
import pandas as pd
import sys
import pickle
from scipy.signal import periodogram

sys.path.append('..')
from utils.features import *
from utils.signal_processing import *

print("Packages Imported")

Packages Imported


In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# settings
data_in = '../data/emg_recordings'

In [13]:
# function to add labels from metadata to data

def add_labels(df, m):
    # first set label column to all nans
    df['label'] = np.nan
    # loop for each action/label
    for i, row in m.iterrows():
        # get start time and label
        t = row[0]
        cat = row[1].strip()
        
        # turn HOLD and STOP labels into corresponding action
        if cat == "HOLD":
            label = m[1].iloc[i-1].strip()
        elif cat == 'STOP':
            label = m[1].iloc[i-2].strip()
        else:
            label = cat
            
        # set all labels passed start time to correct label
        df['label'][df[0] >= t] = label
    return df

In [14]:
ses3 = pd.read_csv(f"{data_in}/test_sess3_data.txt", header=None)
m3 = pd.read_csv(f"{data_in}/test_sess3_metadata.txt", header=None)

df3 = add_labels(ses3, m3)

In [15]:
ses4 = pd.read_csv(f"{data_in}/test_sess4_data.txt", header=None)
m4 = pd.read_csv(f"{data_in}/test_sess4_metadata.txt", header=None)

df4 = add_labels(ses4, m4)

In [16]:
ses5 = pd.read_csv(f"{data_in}/test_sess5_data.txt", header=None)
m5 = pd.read_csv(f"{data_in}/test_sess5_metadata.txt", header=None)

df5 = add_labels(ses5, m5)

In [17]:
# concatenate all sessions
df = pd.concat([df3, df4, df5]).dropna().reset_index(drop=True)

In [18]:
df.head()

,0,1,2,3,4,5,6,7,8,label
0,122155.864712,-77062.468750,-77093.445312,-73484.914062,-73730.742188,-103410.390625,-103418.390625,-173520.750000,-173397.546875,REST
1,122155.868712,-76983.476562,-77014.007812,-73467.031250,-73712.406250,-103027.210938,-103034.437500,-173118.437500,-172994.953125,REST
2,122155.872712,-76935.195312,-76965.304688,-73453.265625,-73698.507812,-102904.882812,-102912.507812,-173069.765625,-172946.484375,REST
3,122155.876712,-77000.687500,-77031.289062,-73466.515625,-73712.500000,-103259.335938,-103266.984375,-173449.359375,-173326.406250,REST
4,122155.880712,-77053.593750,-77084.703125,-73480.265625,-73726.382812,-103445.171875,-103453.062500,-173564.531250,-173441.359375,REST


In [19]:
df['label'].unique()

array(['REST', 'WRIST DOWN', 'SNAP', 'CLENCH FIST', 'WRIST UP'],
      dtype=object)

In [20]:
# metric calculations

def compute_metrics(data):
    
    # time domain
    tfuncs = [mean_absolute_value, slope_sign_changes, root_mean_square]
    
    # freq domain
    freq, power = periodogram(data, 250)
    ffuncs = [median_frequency, mean_frequency]
    
    return [f(data) for f in tfuncs] + [f(power, freq) for f in ffuncs] + average_band_power(power, freq)

In [21]:
## THIS CELL IS FOR TESTING COMPUTE_METRICS

raw_data = df.iloc[0:50]

# if the data is between rest and a motion, don't include it
if len(raw_data['label'].unique()) > 1:
    i += window_size
    #continue

# loop through all pairs of channels and transform, filter, and compute metrics
all_channel_metrics = []
for e in range(1,9,2):
    sig = tripolar_laplacian(raw_data[e], raw_data[e+1])
    filtered_sig = filter_emg(np.array(sig), 250)

    metrics = compute_metrics(filtered_sig)
    
metrics

[103.21974907407392, 23, 116.01697863878724]

In [22]:
# create features based on 0.2 seconds (50 rows) of data points

window_size = 50
i = 0
feats = []
# loop through 50 data points at a time
while i + window_size < len(df):
    # select data of interest
    raw_data = df.iloc[i:i + window_size]
    
    # if the data is between rest and a motion, don't include it
    if len(raw_data['label'].unique()) > 1:
        i += window_size
        continue

    # loop through all pairs of channels and transform, filter, and compute metrics
    all_channel_metrics = []
    for e in range(1,9,2):
        sig = tripolar_laplacian(raw_data[e], raw_data[e+1])
        filtered_sig = filter_emg(np.array(sig), 250)

        metrics = compute_metrics(filtered_sig)

        all_channel_metrics.extend(metrics)

    # add metrics plus label to matrix and go next
    feats.append(all_channel_metrics + [raw_data['label'].unique()[0]])
    i += window_size

In [23]:
feature_df = pd.DataFrame(feats)
feature_df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,23.054868,23,26.879249,5.092542,23,5.949945,100.545968,23,112.540689,103.219749,23,116.016979,REST
1,20.062343,23,22.540444,6.796910,23,8.351373,97.515596,23,109.450591,100.765259,23,113.423511,REST
2,15.540885,23,17.822811,3.516999,23,4.172749,90.467274,23,101.667903,94.108229,23,106.311052,REST
3,12.998037,24,14.932618,5.141250,23,6.221360,68.654617,23,76.694255,73.497742,23,82.659126,REST
4,13.582772,24,15.559570,10.066973,23,11.405492,65.496634,23,73.336346,70.924455,23,79.760386,REST
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,1212.371725,23,1342.899112,18.340115,23,20.513414,1010.647343,23,1123.103268,559.697951,23,625.203057,CLENCH FIST
1404,1208.833108,23,1340.400620,18.367587,23,21.286912,1009.983750,23,1124.214785,557.586899,23,621.912292,CLENCH FIST
1405,1208.159727,23,1339.882094,18.461581,23,21.112132,1009.411446,23,1123.136257,558.443133,23,622.274518,CLENCH FIST
1406,1222.780625,23,1356.351442,17.943304,24,19.967945,1028.985873,24,1134.201384,561.552460,24,623.135654,CLENCH FIST


In [24]:
feature_df[feature_df.columns[-1]].value_counts()

REST           548
WRIST DOWN     218
CLENCH FIST    215
WRIST UP       214
SNAP           213
Name: 12, dtype: int64

In [25]:
# now machine learning

In [32]:
# imports
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [27]:
X = feature_df.copy()
y = X.pop(feature_df.columns[-1])

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100)

In [29]:
# SVM
model = SVC()
model.fit(X_train, y_train)

SVC()

In [30]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.4337121212121212
0.4090909090909091


In [31]:
# defining parameter range
params = {'C': [100, 1000, 10000], 
              'gamma': [0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

clf = GridSearchCV(
        estimator = SVC(),
        param_grid = params,
        cv=5,
        n_jobs=-1,
        verbose=1
    )

clf.fit(X_train, y_train)
best_params = clf.best_params_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


In [32]:
best_params

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

In [33]:
model = SVC(**best_params)
model.fit(X_train, y_train)

print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.9081439393939394
0.6619318181818182


In [34]:
# train final model on full dataset

final_model = SVC(**best_params)
final_model.fit(X, y)

# save trained model
with open('../data/models/SVC.pkl','wb') as f:
    pickle.dump(final_model,f)

# Binary Classification on Ses 9  and 10

In [2]:
data_in = '../data/emg_recordings'

In [6]:
# add labels in different manner

def label(df, m):
    # first set label column to all nans
    df['label'] = np.nan
    # loop for each action/label
    for i, row in m.iterrows():
        # get start time and label
        t = row[0]
        cat = row[1].strip()
            
        # set all labels passed start time to correct label
        df['label'][df[0] >= t] = cat
    return df

In [8]:
ses9 = pd.read_csv(f"{data_in}/test_sess9_data.txt", header=None)
m9 = pd.read_csv(f"{data_in}/test_sess9_metadata.txt", header=None)

df9 = label(ses9, m9)

In [10]:
ses10 = pd.read_csv(f"{data_in}/test_sess10_data.txt", header=None)
m10 = pd.read_csv(f"{data_in}/test_sess10_metadata.txt", header=None)

df10 = label(ses10, m10)

In [19]:
df = pd.concat([df9, df10]).dropna()
df = df[df['label']!='REST'].reset_index()

In [20]:
df

,index,0,1,2,3,4,5,6,7,8,label
0,1508,193908.879273,-13426.044922,-13466.278320,-5925.022949,-6914.467285,221.081100,212.609787,-13538.138672,-13498.576172,CLENCH FIST
1,1510,193908.883101,-13427.743164,-13468.021484,-5806.290527,-6941.602539,221.505783,212.766251,-13539.636719,-13500.140625,CLENCH FIST
2,1511,193908.887101,-13428.749023,-13468.982422,-5924.352051,-6926.112793,221.192856,212.207458,-13540.932617,-13501.325195,CLENCH FIST
3,1512,193908.891101,-13426.223633,-13466.412109,-5937.227051,-6900.631836,221.170517,212.252167,-13538.362305,-13498.844727,CLENCH FIST
4,1513,193908.895101,-13424.725586,-13464.914062,-5927.950684,-6912.143066,221.282272,212.386276,-13536.909180,-13497.301758,CLENCH FIST
...,...,...,...,...,...,...,...,...,...,...,...
10162,11405,194284.001104,-18191.548828,-18234.375000,-10717.459961,-11683.927734,326.827209,321.663940,-18303.173828,-18263.632812,SNAP
10163,11406,194284.005104,-18190.720703,-18233.636719,-10727.629883,-11693.740234,330.023499,324.726135,-18302.369141,-18262.849609,SNAP
10164,11407,194284.009104,-18186.988281,-18229.904297,-10660.418945,-11627.086914,334.583252,329.196503,-18298.703125,-18259.140625,SNAP
10165,11408,194284.013104,-18183.880859,-18226.707031,-10628.612305,-11595.124023,332.169281,327.073090,-18295.505859,-18255.876953,SNAP


In [28]:
# metric calculations

def compute_metrics(data):
    
    # time domain
    tfuncs = [mean_absolute_value, slope_sign_changes, root_mean_square]
    
    # freq domain
    freq, power = periodogram(data, 250)
    ffuncs = [median_frequency, mean_frequency]
    
    return [f(data) for f in tfuncs] + [f(power, freq) for f in ffuncs] + calculate_average_power(power, freq)

In [30]:
# create features based on 0.2 seconds (50 rows) of data points

window_size = 50
i = 0
feats = []
# loop through 50 data points at a time
while i + window_size < len(df):
    # select data of interest
    raw_data = df.iloc[i:i + window_size]
    
    # if the data is between rest and a motion, don't include it
    if len(raw_data['label'].unique()) > 1:
        i += window_size
        continue

    # loop through all pairs of channels and transform, filter, and compute metrics
    all_channel_metrics = []
    for e in range(1,9,2):
        sig = tripolar_laplacian(raw_data[e], raw_data[e+1])
        filtered_sig = filter_emg(np.array(sig), 250)

        metrics = compute_metrics(filtered_sig)

        all_channel_metrics.extend(metrics)

    # add metrics plus label to matrix and go next
    feats.append(all_channel_metrics + [raw_data['label'].unique()[0]])
    i += window_size

In [70]:
feature_df = pd.DataFrame(feats)
feature_df

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1.014232,23,1.219033,60.0,44.414236,0.016414,0.007199,0.033229,0.001612,0.000981,...,23,1.182345,60.0,43.909518,0.016009,0.007221,0.030150,0.001472,0.001055,CLENCH FIST
1,1.290276,23,1.575104,60.0,32.982045,0.047417,0.008057,0.040840,0.002454,0.000469,...,23,1.559089,60.0,32.083251,0.048177,0.007787,0.038174,0.002581,0.000511,CLENCH FIST
2,40.338417,15,54.751055,10.0,11.055355,112.673109,5.785860,0.775096,0.340035,0.299614,...,15,56.152463,10.0,11.037352,118.595401,6.044861,0.800154,0.343659,0.305797,CLENCH FIST
3,41.780050,14,58.317593,5.0,11.890147,128.179392,4.900321,1.160589,0.958466,0.678368,...,14,59.301860,5.0,11.856805,132.566102,5.090212,1.173800,0.980734,0.690603,CLENCH FIST
4,32.214243,14,57.133149,55.0,47.391591,51.748010,20.679571,19.355097,18.183452,18.714198,...,14,57.509779,55.0,47.209913,52.819541,20.849886,19.527545,18.317240,18.876044,CLENCH FIST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,22.299967,24,23.778196,5.0,11.023410,20.404521,1.152928,0.617392,0.234004,0.166738,...,24,23.825978,5.0,11.024265,20.489794,1.156953,0.613879,0.236565,0.168670,SNAP
178,6.226368,21,7.078086,5.0,8.253740,1.914121,0.024889,0.054334,0.005192,0.004552,...,21,7.091948,5.0,8.223537,1.922257,0.025592,0.053085,0.005328,0.004608,SNAP
179,8.621284,19,9.673027,5.0,7.057366,3.610118,0.076430,0.043532,0.010890,0.001722,...,19,9.726251,5.0,7.040237,3.652424,0.076545,0.042292,0.010984,0.001743,SNAP
180,58.441938,8,65.310333,5.0,8.477556,160.746816,7.609788,1.439812,0.618549,0.199313,...,8,67.685500,5.0,8.441867,172.791949,8.090807,1.518860,0.644012,0.204395,SNAP


In [33]:
X = feature_df.copy()
y = X.pop(feature_df.columns[-1])

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100)

In [35]:
# SVM
model = SVC()
model.fit(X_train, y_train)

SVC()

In [36]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.5514705882352942
0.391304347826087


In [45]:
# defining parameter range
params = {'C': [1, 10, 100, 1000, 10000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

clf = GridSearchCV(
        estimator = SVC(),
        param_grid = params,
        cv=5,
        n_jobs=-1,
        verbose=1
    )

clf.fit(X_train, y_train)
best_params = clf.best_params_

Fitting 5 folds for each of 25 candidates, totalling 125 fits


In [46]:
best_params

{'C': 1, 'gamma': 1, 'kernel': 'rbf'}

In [47]:
model = SVC(**best_params)
model.fit(X_train, y_train)

print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

1.0
0.391304347826087


In [72]:
def raw_score(df, feats):
    # get feats
    fdf = df[feats]
    
    # split data
    X = feature_df.copy()
    y = X.pop(feature_df.columns[-1])
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    # train model
    model = SVC()
    model.fit(X_train, y_train)
    
    # report score
    print(model.score(X_train, y_train))
    print(model.score(X_test, y_test))

In [126]:
# no band power
feats = [0,1,2,3,4,10,11,12,13,14,20,21,22,23,24,30,31,32,33,34,40]

raw_score(feature_df, feats)

0.5147058823529411
0.5


In [69]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.5882352941176471
0.3695652173913043


In [133]:
# just band power
feats = [5,6,7,8,9,15,16,17,18,19,25,26,27,28,29,35,36,37,38,39,40]

raw_score(feature_df, feats)

0.5073529411764706
0.5217391304347826


In [145]:
# just time domain
feats = [0,1,2,10,11,12,20,21,22,30,31,32,40]

raw_score(feature_df, feats)

0.5882352941176471
0.5434782608695652


In [154]:
# just freq domain minus band power
feats = [3,4,13,14,23,24,33,34,40]

raw_score(feature_df, feats)

0.5073529411764706
0.45652173913043476


In [ ]:
# it's all bad!

In [ ]:
# I suppose we can use just time domain features for now to make it a faster process at least

In [155]:
# just time domain
feats = [0,1,2,10,11,12,20,21,22,30,31,32,40]

X = feature_df[feats].copy()
y = X.pop(feature_df.columns[-1])
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [156]:
# defining parameter range
params = {'C': [1, 10, 100, 1000, 10000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

clf = GridSearchCV(
        estimator = SVC(),
        param_grid = params,
        cv=5,
        n_jobs=-1,
        verbose=1
    )

clf.fit(X_train, y_train)
best_params = clf.best_params_

Fitting 5 folds for each of 25 candidates, totalling 125 fits


In [157]:
best_params

{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}

In [158]:
model = SVC(**best_params)
model.fit(X_train, y_train)

print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.9705882352941176
0.5434782608695652


In [159]:
# train final model on full dataset

final_model = SVC(**best_params)
final_model.fit(X, y)

# save trained model
with open('../data/models/binary_SVC.pkl','wb') as f:
    pickle.dump(final_model,f)